<a href="https://colab.research.google.com/github/Baah134/Baah134/blob/main/Titanic_Challenge/Titanic_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/DeepLearning/Titanic_Challenge/train.csv'

df = pd.read_csv(path)
df.sample(10)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
611,612,0,3,"Jardin, Mr. Jose Neto",male,NaN,0,0,SOTON/O.Q. 3101305,7.0500,NaN,S
787,788,0,3,"Rice, Master. George Hugh",male,8.0,4,1,382652,29.1250,NaN,Q
838,839,1,3,"Chip, Mr. Chang",male,32.0,0,0,1601,56.4958,NaN,S
616,617,0,3,"Danbom, Mr. Ernst Gilbert",male,34.0,1,1,347080,14.4000,NaN,S
483,484,1,3,"Turkula, Mrs. (Hedwig)",female,63.0,0,0,4134,9.5875,NaN,S
229,230,0,3,"Lefebre, Miss. Mathilde",female,NaN,3,1,4133,25.4667,NaN,S
732,733,0,2,"Knight, Mr. Robert J",male,NaN,0,0,239855,0.0000,NaN,S
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
218,219,1,1,"Bazzani, Miss. Albina",female,32.0,0,0,11813,76.2917,D15,C
739,740,0,3,"Nankoff, Mr. Minko",male,NaN,0,0,349218,7.8958,NaN,S


In [ ]:
df.shape

(891, 12)

In [ ]:
#Dropping data that is either incomplete or irrelevant
df.drop('Name', axis='columns',inplace=True)
df.drop('Cabin', axis='columns',inplace=True)
df.drop('Ticket', axis='columns',inplace=True)
#df.drop('PassengerId', axis='columns',inplace=True)

In [ ]:
df.shape

(891, 9)

In [ ]:
df.iloc[630].Age

80.0

In [ ]:
df.dropna(subset=['Age'], inplace=True)
df.dropna(subset=['Embarked'], inplace=True)

In [ ]:
df.shape

(712, 9)

In [ ]:
df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,male,22.0,1,0,7.2500,S
1,2,1,1,female,38.0,1,0,71.2833,C
2,3,1,3,female,26.0,0,0,7.9250,S
3,4,1,1,female,35.0,1,0,53.1000,S
4,5,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...,...
885,886,0,3,female,39.0,0,5,29.1250,Q
886,887,0,2,male,27.0,0,0,13.0000,S
887,888,1,1,female,19.0,0,0,30.0000,S
889,890,1,1,male,26.0,0,0,30.0000,C


In [ ]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Sex,object
Age,float64
SibSp,int64
Parch,int64
Fare,float64
Embarked,object


In [ ]:
def print_unique_col_values(df):
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}')

In [ ]:
print_unique_col_values(df)

Sex: ['male' 'female']
Embarked: ['S' 'C' 'Q']


In [ ]:
df['Sex'].replace({'female':1,'male':0},inplace=True)

<ipython-input-72-093d512625cc>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Sex'].replace({'female':1,'male':0},inplace=True)
<ipython-input-72-093d512625cc>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Sex'].replace({'female':1,'male':0},inplace=True)


In [ ]:
for col in df:
    print(f'{col}: {df[col].unique()}')

PassengerId: [  1   2   3   4   5   7   8   9  10  11  12  13  14  15  16  17  19  21
  22  23  24  25  26  28  31  34  35  36  38  39  40  41  42  44  45  50
  51  52  53  54  55  57  58  59  60  61  63  64  67  68  69  70  71  72
  73  74  75  76  79  80  81  82  84  85  86  87  89  90  91  92  93  94
  95  97  98  99 100 101 103 104 105 106 107 109 111 112 113 114 115 116
 117 118 119 120 121 123 124 125 126 128 130 131 132 133 134 135 136 137
 138 139 140 142 143 144 145 146 147 148 149 150 151 152 153 154 156 157
 158 161 162 163 164 165 166 168 170 171 172 173 174 175 176 178 179 180
 183 184 185 188 189 190 191 192 193 194 195 196 198 200 201 203 204 205
 206 207 208 209 210 211 212 213 214 216 217 218 219 220 221 222 223 225
 226 227 228 229 231 232 233 234 235 237 238 239 240 243 244 245 246 247
 248 249 250 252 253 254 255 256 258 259 260 262 263 264 266 267 268 269
 270 272 273 274 276 277 279 280 281 282 283 284 286 287 288 289 290 291
 292 293 294 295 297 298 300 303 306 3

In [ ]:
#One-hot encoding for Categorical Columns
df = pd.get_dummies(data=df, columns=['Embarked'], dtype= int)
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch',
       'Fare', 'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [ ]:
df.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Sex,int64
Age,float64
SibSp,int64
Parch,int64
Fare,float64
Embarked_C,int64
Embarked_Q,int64


In [ ]:
cols_to_scale = ['Age','Fare','SibSp','Parch','Pclass']
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df[cols_to_scale] = scaler.fit_transform(df[cols_to_scale])

In [ ]:
df.sample(10)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
514,515,0,1.0,0,0.296306,0.0,0.000000,0.014631,0,0,1
427,428,1,0.5,1,0.233476,0.0,0.000000,0.050749,0,0,1
178,179,0,0.5,0,0.371701,0.0,0.000000,0.025374,0,0,1
137,138,0,0.0,0,0.459663,0.2,0.000000,0.103644,0,0,1
764,765,0,1.0,0,0.195778,0.0,0.000000,0.015176,0,0,1
141,142,1,1.0,1,0.271174,0.0,0.000000,0.015127,0,0,1
436,437,0,1.0,1,0.258608,0.4,0.333333,0.067096,0,0,1
543,544,1,0.5,0,0.396833,0.2,0.000000,0.050749,0,0,1
269,270,1,0.0,1,0.434531,0.0,0.000000,0.264739,0,0,1
296,297,0,1.0,0,0.290023,0.0,0.000000,0.014110,1,0,0


In [ ]:
X = df.drop('Survived',axis='columns')
y = df['Survived']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=5)

In [ ]:
X_train.shape

(569, 10)

In [ ]:
X_train[:10]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
222,223,1.0,0,0.635587,0.0,0.000000,0.015713,0,0,1
694,695,0.0,0,0.748681,0.0,0.000000,0.051822,0,0,1
827,828,0.5,0,0.007288,0.0,0.333333,0.072227,1,0,0
287,288,1.0,0,0.271174,0.0,0.000000,0.015412,0,0,1
678,679,1.0,1,0.535059,0.2,1.000000,0.091543,0,0,1
726,727,0.5,1,0.371701,0.6,0.000000,0.040989,0,0,1
162,163,1.0,0,0.321438,0.0,0.000000,0.015176,0,0,1
748,749,0.0,0,0.233476,0.2,0.000000,0.103644,0,0,1
510,511,1.0,0,0.359135,0.0,0.000000,0.015127,0,1,0
814,815,1.0,0,0.377984,0.0,0.000000,0.015713,0,0,1


In [ ]:
import tensorflow as tf
from tensorflow import keras


model = keras.Sequential([
    keras.layers.Dense(10, input_shape=(10,), activation='relu'),
    keras.layers.Dense(25, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(90, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# opt = keras.optimizers.Adam(learning_rate=0.01)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4712 - loss: 1.7733
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4689 - loss: 0.9286 
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5369 - loss: 0.8535
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5187 - loss: 0.7204
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5259 - loss: 0.8287 
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5227 - loss: 0.7566
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5618 - loss: 0.7133  
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5358 - loss: 0.7924
Epoch 9/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5508 - loss: 0.8517
Epoch 10/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5331 - loss: 0.7278
Epoch 11/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5693 - loss: 0.8632 
Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4710

In [ ]:
model.evaluate(X_test, y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8681 - loss: 0.3865  


[0.4265937805175781, 0.8321678042411804]

In [ ]:
model.save_weights('my_model_weights.weights.h5')


In [ ]:
new_data_path = '/content/drive/MyDrive/DeepLearning/Titanic_Challenge/test.csv'  # Replace with the actual path
new_df = pd.read_csv(new_data_path)

In [ ]:
new_df.sample(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
161,1053,3,"Touma, Master. Georges Youssef",male,7.0,1,1,2650,15.2458,NaN,C
291,1183,3,"Daly, Miss. Margaret Marcella Maggie""""",female,30.0,0,0,382650,6.9500,NaN,Q
310,1202,3,"Cacic, Mr. Jego Grga",male,18.0,0,0,315091,8.6625,NaN,S
98,990,3,"Braf, Miss. Elin Ester Maria",female,20.0,0,0,347471,7.8542,NaN,S
87,979,3,"Badman, Miss. Emily Louisa",female,18.0,0,0,A/4 31416,8.0500,NaN,S
238,1130,2,"Hiltunen, Miss. Marta",female,18.0,1,1,250650,13.0000,NaN,S
381,1273,3,"Foley, Mr. Joseph",male,26.0,0,0,330910,7.8792,NaN,Q
369,1261,2,"Pallas y Castello, Mr. Emilio",male,29.0,0,0,SC/PARIS 2147,13.8583,NaN,C
401,1293,2,"Gale, Mr. Harry",male,38.0,1,0,28664,21.0000,NaN,S
325,1217,3,"Assam, Mr. Ali",male,23.0,0,0,SOTON/O.Q. 3101309,7.0500,NaN,S


In [ ]:
#Dropping data that is either incomplete or irrelevant
new_df.drop('Name', axis='columns',inplace=True)
new_df.drop('Cabin', axis='columns',inplace=True)
new_df.drop('Ticket', axis='columns',inplace=True)


new_df.fillna(0, inplace=True)

new_df['Sex'].replace({'female':1,'male':0},inplace=True)



cols_to_scale = ['Age','Fare','SibSp','Parch','Pclass',"PassengerId"]
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
new_df[cols_to_scale] = scaler.fit_transform(new_df[cols_to_scale])

<ipython-input-88-2bbe0ba92163>:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new_df['Sex'].replace({'female':1,'male':0},inplace=True)
<ipython-input-88-2bbe0ba92163>:9: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  new_df['Sex'].replace({'female':1,'male':0},inplace=True)


In [ ]:
new_df.dtypes

,0
PassengerId,float64
Pclass,float64
Sex,int64
Age,float64
SibSp,float64
Parch,float64
Fare,float64
Embarked,object


In [ ]:
#One-hot encoding for Categorical Columns
new_df = pd.get_dummies(data=new_df, columns=['Embarked'], dtype= int)
new_df.columns

Index(['PassengerId', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked_C', 'Embarked_Q', 'Embarked_S'],
      dtype='object')

In [ ]:
new_df.sample(10)

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
282,0.676259,1.0,1,0.000000,0.000,0.000000,0.015127,0,1,0
127,0.304556,1.0,1,0.000000,0.250,0.000000,0.045381,0,1,0
206,0.494005,1.0,1,0.460526,0.000,0.000000,0.015127,0,1,0
243,0.582734,1.0,0,0.000000,0.000,0.000000,0.015395,0,0,1
230,0.551559,0.5,0,0.184211,0.000,0.000000,0.126872,0,0,1
305,0.731415,0.0,1,0.842105,0.125,0.111111,0.051822,0,0,1
253,0.606715,1.0,0,0.315789,0.000,0.000000,0.018201,0,0,1
341,0.817746,1.0,0,0.421053,0.000,0.000000,0.014794,0,0,1
197,0.472422,1.0,1,0.236842,0.000,0.000000,0.015176,0,0,1
329,0.788969,0.5,0,0.276316,0.000,0.000000,0.025374,0,0,1


In [ ]:
model.load_weights('my_model_weights.weights.h5')
predictions = model.predict(new_df)
predicted_labels = (predictions > 0.5).astype(int)


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [ ]:
nnew_df = pd.read_csv(new_data_path)

In [ ]:
predictions_df = pd.DataFrame({'PassengerId': nnew_df['PassengerId'], 'Survived': predicted_labels.ravel()}) #Flatten predicted_labels to 1D
predictions_df.to_csv('predictions.csv', index=False)  # Set index=False to avoid saving the index
from google.colab import files
files.download('predictions.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>